This notebook processes and merges various clinical risk factor datasets (i.e. AllClinical00_SAS, Enrollees_SAS, kXR_SQ_BU00_SAS) and the TKR labels in the full cohort into a single dataset. The resulting dataset is saved to `output/combined_clinical_data.csv` and contains knee-level observations with: 
- knee-level features 
- patient-level features (i.e. both knees of same patient share the same feature values) 
- and knee-level features describing the patient's other knee (e.g. right knee features joined to left knee observation) 

This notebook also outputs a single variable description file from the corresponding variable description files of each dataset that Cem parsed, saved to `output/combined_clinical_variable_desc.csv`

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
import time
from datetime import date, timedelta, datetime

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [2]:
# mount our google drive onto colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# set that folder to be our working directory 
import os 
os.chdir("/content/gdrive/My Drive/oai-xray-tkr-klg")

# Load & merge clinical risk factor data onto knee-level cohort 

In [4]:
COHORT = 'data/KR_cohort_2021_10_28_with_splits_v2.csv'
DATA_CLINICAL = 'data/2020_Capstone/Clinical/AllClinical00_SAS/AllClinical00.txt'
DATA_ENROLLEES = 'data/2020_Capstone/Clinical/Enrollees_SAS/Enrollees.txt'
DATA_kXR_SQ_BU00 = 'data/2020_Capstone/Clinical/kXR_SQ_BU00_SAS/kxr_sq_bu00.txt'

In [5]:
# cohort is at the knee level 
cohort = pd.read_csv(COHORT)
cohort = cohort.rename(columns={'id': 'ID', 'Side': 'SIDE', 'KR_Label': 'KR_LABEL'})
cohort = cohort.drop(['DATE', 'DAYS'], axis=1)
cohort['SIDE'] = np.where(cohort['SIDE']=='RIGHT', 1, 2)
assert (cohort.groupby(['ID', 'SIDE']).size() == 1).all()
cohort.info()
cohort.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8957 entries, 0 to 8956
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Knee_ID             8957 non-null   object 
 1   ID                  8957 non-null   int64  
 2   SIDE                8957 non-null   int64  
 3   KR_LABEL            8957 non-null   int64  
 4   Xray_HDF5_Filename  8957 non-null   object 
 5   Xray_HDF5_Exists    8957 non-null   int64  
 6   Eligible            8957 non-null   int64  
 7   Split               8932 non-null   object 
 8   TrainFold           6695 non-null   float64
dtypes: float64(1), int64(5), object(3)
memory usage: 629.9+ KB


,Knee_ID,ID,SIDE,KR_LABEL,Xray_HDF5_Filename,Xray_HDF5_Exists,Eligible,Split,TrainFold
0,9002430_RIGHT,9002430,1,1,9002430_00m_RIGHT_KNEE.hdf5,1,1,train,2.0
1,9014797_RIGHT,9014797,1,1,9014797_00m_RIGHT_KNEE.hdf5,1,1,train,3.0
2,9015363_RIGHT,9015363,1,1,9015363_00m_RIGHT_KNEE.hdf5,1,1,train,1.0
3,9028418_RIGHT,9028418,1,1,9028418_00m_RIGHT_KNEE.hdf5,1,1,train,1.0
4,9030718_RIGHT,9030718,1,1,9030718_00m_RIGHT_KNEE.hdf5,1,1,train,2.0


In [6]:
# clinical data is at the patient level 
clinical = pd.read_csv(DATA_CLINICAL, sep="|")
clinical = clinical.drop('VERSION', axis=1)
assert (clinical.groupby(['ID']).size() == 1).all()
clinical.info()
clinical.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4796 entries, 0 to 4795
Columns: 1225 entries, ID to V00WKHR7CV
dtypes: float64(1177), int64(31), object(17)
memory usage: 44.8+ MB


,ID,P01RXRKOA,P01LXRKOA,V00URINHR1,V00URINHR2,V00BLDHRS1,V00BLDHRS2,V00SEAQHR1,V00SEAQHR2,V00PLAQHR1,V00PLAQHR2,V00URINOB1,V00URINOB2,V00VOID1,V00VOID2,V00ILLPWK1,V00ILLPWK2,V00HOURSP1,V00HOURSP2,V00QOVP1,V00QOVP2,V00VCOLL1,V00VCOLL2,V00HEMAT1,V00HEMAT2,V00VEIN1,V00VEIN2,V00MULTST1,V00MULTST2,V00EXCESS1,V00EXCESS2,V00LEAKAG1,V00LEAKAG2,V00OTHVP1,V00OTHVP2,V00BLDRAW1,V00BLDRAW2,V00HRSUC1,V00HRSUC2,P01SVXRRID,P01SVXRRKR,P01SVXRLKR,P01SVRKOST,P01SVLKOST,P01SVRKJSM,P01SVLKJSM,P01SVRKJSL,P01SVLKJSL,P01SVRKMI,P01SVLKMI,P01SVXRELK,P01XRKOA,P01RXRKOA2,P01LXRKOA2,V00XRBCODE,V00XROSFLR,V00XRSCFLR,V00XRCYFLR,V00XRJSLR,V00XRCHLR,V00XROSTLR,V00XRSCTLR,V00XRCYTLR,V00XRATTLR,V00XROSFMR,V00XRSCFMR,V00XRCYFMR,V00XRJSMR,V00XRCHMR,V00XROSTMR,V00XRSCTMR,V00XRCYTMR,V00XRATTMR,V00XRKLR,V00XROSFML,V00XRSCFML,V00XRCYFML,V00XRJSML,V00XRCHML,V00XROSTML,V00XRSCTML,V00XRCYTML,V00XRATTML,V00XRKLL,V00XROSFLL,V00XRSCFLL,V00XRCYFLL,V00XRJSLL,V00XRCHLL,V00XROSTLL,V00XRSCTLL,V00XRCYTLL,V00XRATTLL,P01OAGRDR,V00BLDCOLL,V00UCDATE2,V00URNCOLL,V00PDATE2,V00PDATE1,V00UCDATE1,P01OAGRDL,V00BLSURD2,V00URSURD2,V00BLSURD1,V00URSURD1,V00MRSEQNR,V00MRSEQNL,V00SERUM,V00EDTA,V00CITRATE,P02KPN,P01KPNREV,P01KPNREVY,P01KPNR12,P01KPNR12M,P01KPNLEV,P01KPNLEVY,P01KPNL12,P01KPNL12M,P01KPACT30,P01HPNR12,P01HPNRIL,P01HPNROL,P01HPNRFL,P01HPNRB,P01HPNRLB,P01HPNRDK,P01HPNL12,P01HPNLIL,P01HPNLOL,P01HPNLFL,P01HPNLB,P01HPNLLB,P01HPNLDK,P01BP30,P01BP30OFT,P01BPBAD,P01BPUB,P01BPMB,P01BPLB,P01BPB,P01BPDK,P01OJPNRS,P01OJPNLS,P01OJPNRE,P01OJPNLE,P01OJPNRW,P01OJPNLW,P01OJPNRH,P01OJPNLH,P01OJPNRA,P01OJPNLA,P01OJPNRF,P01OJPNLF,P01OJPNNK,P01OJPNNO,P01TMJE6M,P01TMJEPN1,P01TMJE30D,P01TMJE30A,P01TMJF6M,P01TMJFPN1,P01TMJF30D,P01TMJF30A,V00SF1,V00SF2,V00SF3,V00SF4,V00SF5,V00SF6,V00SF7,V00SF8,V00SF9,V00SF10,V00SF11,V00SF12,V00WPRKN1,V00WPRKN2,V00WPRKN3,V00WPRKN4,V00WPRKN5,V00KPRKN1,V00KPRKN2,V00KPRKN3,V00P7RKFR,V00WSRKN1,V00WSRKN2,V00KSXRKN1,V00KSXRKN2,V00KSXRKN3,V00KSXRKN4,V00KSXRKN5,V00DIRKN1,V00DIRKN2,V00DIRKN3,V00DIRKN4,V00DIRKN5,V00DIRKN6,V00DIRKN7,V00DIRKN8,V00DIRKN9,V00DIRKN10,V00DIRKN11,V00DIRKN12,V00DIRKN13,V00DIRKN14,V00DIRKN15,V00DIRKN16,V00DIRKN17,V00WPLKN1,V00WPLKN2,V00WPLKN3,V00WPLKN4,V00WPLKN5,V00KPLKN1,V00KPLKN2,V00KPLKN3,V00P7LKFR,V00WSLKN1,V00WSLKN2,V00KSXLKN1,V00KSXLKN2,V00KSXLKN3,V00KSXLKN4,V00KSXLKN5,V00DILKN1,V00DILKN2,V00DILKN3,V00DILKN4,V00DILKN5,V00DILKN6,V00DILKN7,V00DILKN8,V00DILKN9,V00DILKN10,V00DILKN11,V00DILKN12,V00DILKN13,V00DILKN14,V00DILKN15,V00DILKN16,V00DILKN17,V00KOOSFX1,V00KOOSFX2,V00KOOSFX3,V00KOOSFX4,V00KOOSFX5,V00KQOL1,V00KQOL2,V00KQOL3,V00KQOL4,V00KGLRS,P02KPNRCV,P02KPNLCV,P01KPR30CV,P01KPL30CV,P01KPACTCV,P01HPR12CV,P01HPL12CV,P01BPTOT,V00HSPSS,V00HSMSS,V00WOMKPR,V00KOOSKPR,V00WOMSTFR,V00KOOSYMR,V00WOMADLR,V00WOMKPL,V00KOOSKPL,V00WOMSTFL,V00KOOSYML,V00WOMADLL,V00KOOSFSR,V00KOOSQOL,P01SXKOA,P01KPR12CV,P01KPL12CV,P01KPA30CV,V00P7RKRCV,V00P7LKRCV,P01PMRKRCV,P01PMLKRCV,P01TJE30CV,P01TJF30CV,P01TJE30WC,P01TJF30WC,V00WOMTSL,V00WOMTSR,P01RKSX,P01LKSX,P01LSXKOA,P01RSXKOA,P01BPDAYCV,P01BPBEDCV,P01BPACTCV,P01KSX,P01BL12SXL,P01BL12SXR,P01KPACDCV,P01LKP30CV,P01RKP30CV,P02KPMED,P02KINJ,P02KSURG,P02CNCR3,P02CNC3,P02CNC4,P02CNC13,P02CNC14,P02CNC15,P02CNMEL,P02CSY,P02CMN,P02CMDK,P01RAIA,P01RAJS1HR,P01RAFT6R,P01RAFT6L,P01RAKN6R,P01RAKN6L,P01RA3YES,P01RA1YES,P01RARLYES,P01RATEST,P01RATOT3,P01ARTHOTH,P01ARTDOC,P01KPMED,P01INJR,P01INJR1,P01INJR2,P01INJR3,P01KSURGR,P01KRSR,P01KRSRA,P01ARTR,P01ARTR1,P01ARTR2,P01ARTR3,P01ARTRINJ,P01MENR,P01MENR1,P01MENR2,P01MENR3,P01MENRINJ,P01LRR,P01LRR1,P01LRR2,P01LRR3,P01OTSURGR,P01OTSR1,P01OTSR2,P01OTSR3,P01OTSRINJ,P01INJL,P01INJL1,P01INJL2,P01INJL3,P01KSURGL,P01KRSL,P01KRSLA,P01ARTL,P01ARTL1,P01ARTL2,P01ARTL3,P01ARTLINJ,P01MENL,P01MENL1,P01MENL2,P01MENL3,P01MENLINJ,P01LRL,P01LRL1,P01LRL2,P01LRL3,P01OTSURGL,P01OTSL1,P01OTSL2,P01OTSL3,P01OTSLINJ,P01HRS,P01HRSR,P01HRSROA,P01HRSRDK,P01HRSL,P01HRSLOA,P01HRSLDK,P01PREGEV,P01OVREM,P01OV1AGE,P01OV2AGE,P01HYS,P01HYSAGE,P01MENSTR,V00HRTAT,V00HRTFAIL,V00BYPLEG,V00STROKE,V00ASTHMA,V00ASTMEDS,V00AMWHEN,V00LUNG,

In [7]:
# enrollees data is at the patient level
enrollees = pd.read_csv(DATA_ENROLLEES, sep="|")
enrollees = enrollees.drop('VERSION', axis=1)
enrollees['P02RACE'] = enrollees['P02RACE'].fillna(0).astype(int) # assign 5 nans to other non-white 
assert (enrollees.groupby('ID').size() == 1).all()
enrollees.info()
enrollees.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4796 entries, 0 to 4795
Data columns (total 59 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          4796 non-null   int64  
 1   P02HISP     4794 non-null   float64
 2   P02RACE     4796 non-null   int64  
 3   P02SEX      4796 non-null   int64  
 4   V00CHRTHLF  4796 non-null   int64  
 5   V00COHORT   4796 non-null   int64  
 6   V00IMAGESA  4796 non-null   int64  
 7   V00IMAGESB  4796 non-null   int64  
 8   V00IMAGESC  4796 non-null   int64  
 9   V00IMAGESD  4796 non-null   int64  
 10  V00IMAGESE  4796 non-null   int64  
 11  V00IMAGESF  4796 non-null   int64  
 12  V00IMAGESG  4796 non-null   int64  
 13  V00SITE     4796 non-null   object 
 14  V01HADINTV  4796 non-null   int64  
 15  V01IMAGESA  4796 non-null   int64  
 16  V01IMAGESB  4796 non-null   int64  
 17  V01IMAGESC  4543 non-null   float64
 18  V01IMAGESD  4796 non-null   int64  
 19  V01IMAGESE  4566 non-null  

,ID,P02HISP,P02RACE,P02SEX,V00CHRTHLF,V00COHORT,V00IMAGESA,V00IMAGESB,V00IMAGESC,V00IMAGESD,V00IMAGESE,V00IMAGESF,V00IMAGESG,V00SITE,V01HADINTV,V01IMAGESA,V01IMAGESB,V01IMAGESC,V01IMAGESD,V01IMAGESE,V01IMAGESF,V01IMAGESG,V02IMAGESD,V03IMAGESA,V03IMAGESB,V03IMAGESC,V03IMAGESD,V03IMAGESE,V03IMAGESF,V03IMAGESG,V04IMAGESG,V05IMAGESA,V05IMAGESB,V05IMAGESC,V05IMAGESD,V05IMAGESE,V05IMAGESF,V05IMAGESG,V06IMAGESA,V06IMAGESB,V06IMAGESC,V06IMAGESD,V06IMAGESE,V06IMAGESF,V06IMAGESG,V08IMAGESA,V08IMAGESB,V08IMAGESC,V08IMAGESD,V08IMAGESE,V08IMAGESF,V08IMAGESG,V10IMAGESA,V10IMAGESB,V10IMAGESC,V10IMAGESD,V10IMAGESE,V10IMAGESF,V10IMAGESG
0,9000099,0.0,1,1,2,1,0,0,0,0,3,0,3,B,2,0,0,0.0,0,3.0,0.0,3.0,0.0,0,0.0,0.0,0.0,3.0,0.0,3.0,1.0,0,0.0,0.0,0.0,3.0,0.0,3.0,0,0.0,0.0,0.0,1.0,0.0,1.0,0,0.0,0.0,0.0,1.0,0.0,1.0,0,0.0,0.0,0.0,1.0,0.0,1.0
1,9000296,0.0,1,1,1,2,3,0,3,0,0,0,0,D,0,3,0,3.0,0,0.0,0.0,0.0,0.0,3,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,3.0,0.0,0.0,0.0,0.0,3,0.0,1.0,0.0,0.0,0.0,0.0,3,0.0,1.0,0.0,0.0,0.0,0.0,3,0.0,1.0,0.0,0.0,0.0,0.0
2,9000622,0.0,1,2,2,1,0,0,0,0,3,0,0,B,0,0,0,0.0,0,3.0,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0
3,9000798,0.0,1,1,1,1,0,0,3,3,0,0,0,D,1,0,0,3.0,3,0.0,0.0,0.0,1.0,0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0,0.0,3.0,3.0,0.0,0.0,0.0,0,0.0,1.0,1.0,0.0,0.0,0.0,0,0.0,1.0,1.0,0.0,0.0,0.0,0,0.0,1.0,1.0,0.0,0.0,0.0
4,9001104,0.0,1,2,2,2,0,0,0,0,3,0,0,B,0,0,0,0.0,0,3.0,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0


In [8]:
# there are multiple rows per knee with diff READPRJ, we filter for those with READPRJ=15
kxr = pd.read_csv(DATA_kXR_SQ_BU00, sep='|')
kxr = kxr.query("READPRJ==15")
kxr = kxr.drop(['READPRJ', 'VERSION'], axis=1)
assert (kxr.groupby(['ID', 'SIDE']).size() == 1).all()

# join the same patient's other knee info as another set of features 
kxr_opposite = kxr.assign(OTHERSIDE=lambda x: np.where(x['SIDE']==1, 2, 1))
kxr_opposite = kxr_opposite.drop('SIDE', axis=1).set_index(['ID', 'OTHERSIDE'])
kxr_opposite.columns = [f'OTHERKNEE_{col}' for col in kxr_opposite.columns.tolist()]
kxr_opposite = kxr_opposite.reset_index()

kxr = kxr.set_index(['ID', 'SIDE'])
kxr.columns = [f'KNEE_{col}' for col in kxr.columns.tolist()]
kxr = kxr.reset_index()
kxr = kxr.merge(kxr_opposite, how='left', left_on=['ID', 'SIDE'], right_on=['ID', 'OTHERSIDE'])

kxr.info()
kxr.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8982 entries, 0 to 8981
Data columns (total 43 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    8982 non-null   int64  
 1   SIDE                  8982 non-null   int64  
 2   KNEE_V00BARCDBU       8980 non-null   float64
 3   KNEE_V00XROSFM        5507 non-null   float64
 4   KNEE_V00XRSCFM        5506 non-null   float64
 5   KNEE_V00XRCYFM        5506 non-null   float64
 6   KNEE_V00XRJSM         8921 non-null   float64
 7   KNEE_V00XRCHM         5506 non-null   float64
 8   KNEE_V00XROSTM        5507 non-null   float64
 9   KNEE_V00XRSCTM        5506 non-null   float64
 10  KNEE_V00XRCYTM        5506 non-null   float64
 11  KNEE_V00XRATTM        5506 non-null   float64
 12  KNEE_V00XRKL          8921 non-null   float64
 13  KNEE_V00XROSFL        5507 non-null   float64
 14  KNEE_V00XRSCFL        5506 non-null   float64
 15  KNEE_V00XRCYFL       

,ID,SIDE,KNEE_V00BARCDBU,KNEE_V00XROSFM,KNEE_V00XRSCFM,KNEE_V00XRCYFM,KNEE_V00XRJSM,KNEE_V00XRCHM,KNEE_V00XROSTM,KNEE_V00XRSCTM,KNEE_V00XRCYTM,KNEE_V00XRATTM,KNEE_V00XRKL,KNEE_V00XROSFL,KNEE_V00XRSCFL,KNEE_V00XRCYFL,KNEE_V00XRJSL,KNEE_V00XRCHL,KNEE_V00XROSTL,KNEE_V00XRSCTL,KNEE_V00XRCYTL,KNEE_V00XRATTL,OTHERSIDE,OTHERKNEE_V00BARCDBU,OTHERKNEE_V00XROSFM,OTHERKNEE_V00XRSCFM,OTHERKNEE_V00XRCYFM,OTHERKNEE_V00XRJSM,OTHERKNEE_V00XRCHM,OTHERKNEE_V00XROSTM,OTHERKNEE_V00XRSCTM,OTHERKNEE_V00XRCYTM,OTHERKNEE_V00XRATTM,OTHERKNEE_V00XRKL,OTHERKNEE_V00XROSFL,OTHERKNEE_V00XRSCFL,OTHERKNEE_V00XRCYFL,OTHERKNEE_V00XRJSL,OTHERKNEE_V00XRCHL,OTHERKNEE_V00XROSTL,OTHERKNEE_V00XRSCTL,OTHERKNEE_V00XRCYTL,OTHERKNEE_V00XRATTL
0,9000099,1,1.660084e+10,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,1.660084e+10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,2.0,2.0,0.0,2.0,0.0,1.0,2.0,0.0,0.0
1,9000099,2,1.660084e+10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,2.0,2.0,0.0,2.0,0.0,1.0,2.0,0.0,0.0,2,1.660084e+10,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,9000296,1,1.660018e+10,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.660018e+10,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,9000296,2,1.660018e+10,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,1.660018e+10,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9000622,1,1.660083e+10,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1.660083e+10,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [9]:
# merge into one dataframe 
df = cohort\
    .merge(clinical, how='left', on='ID')\
    .merge(enrollees, how='left', on='ID')\
    .merge(kxr, how='left', on=['ID', 'SIDE'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8957 entries, 0 to 8956
Columns: 1332 entries, Knee_ID to OTHERKNEE_V00XRATTL
dtypes: float64(1256), int64(55), object(21)
memory usage: 91.1+ MB


,Knee_ID,ID,SIDE,KR_LABEL,Xray_HDF5_Filename,Xray_HDF5_Exists,Eligible,Split,TrainFold,P01RXRKOA,P01LXRKOA,V00URINHR1,V00URINHR2,V00BLDHRS1,V00BLDHRS2,V00SEAQHR1,V00SEAQHR2,V00PLAQHR1,V00PLAQHR2,V00URINOB1,V00URINOB2,V00VOID1,V00VOID2,V00ILLPWK1,V00ILLPWK2,V00HOURSP1,V00HOURSP2,V00QOVP1,V00QOVP2,V00VCOLL1,V00VCOLL2,V00HEMAT1,V00HEMAT2,V00VEIN1,V00VEIN2,V00MULTST1,V00MULTST2,V00EXCESS1,V00EXCESS2,V00LEAKAG1,V00LEAKAG2,V00OTHVP1,V00OTHVP2,V00BLDRAW1,V00BLDRAW2,V00HRSUC1,V00HRSUC2,P01SVXRRID,P01SVXRRKR,P01SVXRLKR,P01SVRKOST,P01SVLKOST,P01SVRKJSM,P01SVLKJSM,P01SVRKJSL,P01SVLKJSL,P01SVRKMI,P01SVLKMI,P01SVXRELK,P01XRKOA,P01RXRKOA2,P01LXRKOA2,V00XRBCODE,V00XROSFLR,V00XRSCFLR,V00XRCYFLR,V00XRJSLR,V00XRCHLR,V00XROSTLR,V00XRSCTLR,V00XRCYTLR,V00XRATTLR,V00XROSFMR,V00XRSCFMR,V00XRCYFMR,V00XRJSMR,V00XRCHMR,V00XROSTMR,V00XRSCTMR,V00XRCYTMR,V00XRATTMR,V00XRKLR,V00XROSFML,V00XRSCFML,V00XRCYFML,V00XRJSML,V00XRCHML,V00XROSTML,V00XRSCTML,V00XRCYTML,V00XRATTML,V00XRKLL,V00XROSFLL,V00XRSCFLL,V00XRCYFLL,V00XRJSLL,V00XRCHLL,V00XROSTLL,V00XRSCTLL,V00XRCYTLL,V00XRATTLL,P01OAGRDR,V00BLDCOLL,V00UCDATE2,V00URNCOLL,V00PDATE2,V00PDATE1,V00UCDATE1,P01OAGRDL,V00BLSURD2,V00URSURD2,V00BLSURD1,V00URSURD1,V00MRSEQNR,V00MRSEQNL,V00SERUM,V00EDTA,V00CITRATE,P02KPN,P01KPNREV,P01KPNREVY,P01KPNR12,P01KPNR12M,P01KPNLEV,P01KPNLEVY,P01KPNL12,P01KPNL12M,P01KPACT30,P01HPNR12,P01HPNRIL,P01HPNROL,P01HPNRFL,P01HPNRB,P01HPNRLB,P01HPNRDK,P01HPNL12,P01HPNLIL,P01HPNLOL,P01HPNLFL,P01HPNLB,P01HPNLLB,P01HPNLDK,P01BP30,P01BP30OFT,P01BPBAD,P01BPUB,P01BPMB,P01BPLB,P01BPB,P01BPDK,P01OJPNRS,P01OJPNLS,P01OJPNRE,P01OJPNLE,P01OJPNRW,P01OJPNLW,P01OJPNRH,P01OJPNLH,P01OJPNRA,P01OJPNLA,P01OJPNRF,P01OJPNLF,P01OJPNNK,P01OJPNNO,P01TMJE6M,P01TMJEPN1,P01TMJE30D,P01TMJE30A,P01TMJF6M,P01TMJFPN1,P01TMJF30D,P01TMJF30A,V00SF1,V00SF2,V00SF3,V00SF4,V00SF5,V00SF6,V00SF7,V00SF8,V00SF9,V00SF10,V00SF11,V00SF12,V00WPRKN1,V00WPRKN2,V00WPRKN3,V00WPRKN4,V00WPRKN5,V00KPRKN1,V00KPRKN2,V00KPRKN3,V00P7RKFR,V00WSRKN1,V00WSRKN2,V00KSXRKN1,V00KSXRKN2,V00KSXRKN3,V00KSXRKN4,V00KSXRKN5,V00DIRKN1,V00DIRKN2,V00DIRKN3,V00DIRKN4,V00DIRKN5,V00DIRKN6,V00DIRKN7,V00DIRKN8,V00DIRKN9,V00DIRKN10,V00DIRKN11,V00DIRKN12,V00DIRKN13,V00DIRKN14,V00DIRKN15,V00DIRKN16,V00DIRKN17,V00WPLKN1,V00WPLKN2,V00WPLKN3,V00WPLKN4,V00WPLKN5,V00KPLKN1,V00KPLKN2,V00KPLKN3,V00P7LKFR,V00WSLKN1,V00WSLKN2,V00KSXLKN1,V00KSXLKN2,V00KSXLKN3,V00KSXLKN4,V00KSXLKN5,V00DILKN1,V00DILKN2,V00DILKN3,V00DILKN4,V00DILKN5,V00DILKN6,V00DILKN7,V00DILKN8,V00DILKN9,V00DILKN10,V00DILKN11,V00DILKN12,V00DILKN13,V00DILKN14,V00DILKN15,V00DILKN16,V00DILKN17,V00KOOSFX1,V00KOOSFX2,V00KOOSFX3,V00KOOSFX4,V00KOOSFX5,V00KQOL1,V00KQOL2,V00KQOL3,V00KQOL4,V00KGLRS,P02KPNRCV,P02KPNLCV,P01KPR30CV,P01KPL30CV,P01KPACTCV,P01HPR12CV,P01HPL12CV,P01BPTOT,V00HSPSS,V00HSMSS,V00WOMKPR,V00KOOSKPR,V00WOMSTFR,V00KOOSYMR,V00WOMADLR,V00WOMKPL,V00KOOSKPL,V00WOMSTFL,V00KOOSYML,V00WOMADLL,V00KOOSFSR,V00KOOSQOL,P01SXKOA,P01KPR12CV,P01KPL12CV,P01KPA30CV,V00P7RKRCV,V00P7LKRCV,P01PMRKRCV,P01PMLKRCV,P01TJE30CV,P01TJF30CV,P01TJE30WC,P01TJF30WC,V00WOMTSL,V00WOMTSR,P01RKSX,P01LKSX,P01LSXKOA,P01RSXKOA,P01BPDAYCV,P01BPBEDCV,P01BPACTCV,P01KSX,P01BL12SXL,P01BL12SXR,P01KPACDCV,P01LKP30CV,P01RKP30CV,P02KPMED,P02KINJ,P02KSURG,P02CNCR3,P02CNC3,P02CNC4,P02CNC13,P02CNC14,P02CNC15,P02CNMEL,P02CSY,P02CMN,P02CMDK,P01RAIA,P01RAJS1HR,P01RAFT6R,P01RAFT6L,P01RAKN6R,P01RAKN6L,P01RA3YES,P01RA1YES,P01RARLYES,P01RATEST,P01RATOT3,P01ARTHOTH,P01ARTDOC,P01KPMED,P01INJR,P01INJR1,P01INJR2,P01INJR3,P01KSURGR,P01KRSR,P01KRSRA,P01ARTR,P01ARTR1,P01ARTR2,P01ARTR3,P01ARTRINJ,P01MENR,P01MENR1,P01MENR2,P01MENR3,P01MENRINJ,P01LRR,P01LRR1,P01LRR2,P01LRR3,P01OTSURGR,P01OTSR1,P01OTSR2,P01OTSR3,P01OTSRINJ,P01INJL,P01INJL1,P01INJL2,P01INJL3,P01KSURGL,P01KRSL,P01KRSLA,P01ARTL,P01ARTL1,P01ARTL2,P01ARTL3,P01ARTLINJ,P01MENL,P01MENL1,P01MENL2,P01MENL3,P01MENLINJ,P01LRL,P01LRL1,P01LRL2,P01LRL3,P01OTSURGL,P01OTSL1,P01OTSL2,P01OTSL3,P01OTSLINJ,P01HRS,P01HRSR,P01HRSROA,P01HRSRDK,P01HRSL,P01HRSLOA,P01HRSLDK,P01PREGEV,P01OVREM,P01OV1AGE,P01OV2AGE,P01HYS,P01HYSAGE,P01MEN

# Load and Merge Variable Descriptions

In [10]:
DESC_CLINICAL = 'data/2020_Capstone/Clinical/AllClinical00_SAS/VariableInformation.xls'
DESC_ENROLLEES = 'data/2020_Capstone/Clinical/Enrollees_SAS/VariableInformation.csv'
DESC_kXR_SQ_BU00 = 'data/2020_Capstone/Clinical/kXR_SQ_BU00_SAS/VariableInformation.csv'

In [11]:
desc_clinical = pd.read_excel(DESC_CLINICAL)
desc_clinical['Use in Analysis'] = desc_clinical['Use in Analysis'].fillna(1) 
desc_clinical = desc_clinical[~desc_clinical['VariableName'].isin(['ID', 'VERSION'])]
desc_clinical.info()
desc_clinical.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1224 entries, 2 to 1225
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   VariableName     1224 non-null   object 
 1   Attribute        1224 non-null   object 
 2   Use in Analysis  1224 non-null   float64
dtypes: float64(1), object(2)
memory usage: 38.2+ KB


,VariableName,Attribute,Use in Analysis
2,P01RXRKOA,SV:Right knee baseline radiographic OA (defini...,1.0
3,P01LXRKOA,SV:Left knee baseline radiographic OA (definit...,1.0
4,V00URINHR1,EV:Urine collection: time collected (first col...,0.0
5,V00URINHR2,EV:Urine collection: time collected (repeat co...,0.0
6,V00BLDHRS1,EV:Phlebotomy: time venipuncture completed (fi...,0.0


In [12]:
desc_enrollees = pd.read_csv(DESC_ENROLLEES, delimiter=',')
desc_enrollees = desc_enrollees.drop('Unnamed: 0', axis=1)
desc_enrollees = desc_enrollees.rename(columns={'liste': 'VariableName', 'attribs': 'Attribute'})
desc_enrollees['Use in Analysis'] = np.where(desc_enrollees['VariableName'].isin(
    ['P02HISP', 'P02RACE', 'P02SEX']), 1, 0)
desc_enrollees = desc_enrollees[~desc_enrollees['VariableName'].isin(['ID', 'VERSION'])]
desc_enrollees.info()
desc_enrollees.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 2 to 59
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   VariableName     58 non-null     object
 1   Attribute        58 non-null     object
 2   Use in Analysis  58 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.8+ KB


,VariableName,Attribute,Use in Analysis
2,P02HISP,P02HISP,1
3,P02RACE,P02RACE,1
4,P02SEX,P02SEX,1
5,V00CHRTHLF,V00CHRTHLF,0
6,V00COHORT,V00COHORT,0


In [13]:
desc_kxr = pd.read_csv(DESC_kXR_SQ_BU00, delimiter=',')
desc_kxr = desc_kxr.rename(columns={'liste': 'VariableName', 'attribs': 'Attribute'})
desc_kxr['Use in Analysis'] = np.where(desc_kxr['VariableName'].isin(
    ['V00BARCDBU']), 0, 1)
desc_kxr = desc_kxr[~desc_kxr['VariableName'].isin(['ID', 'SIDE', 'READPRJ', 'VERSION'])]

desc_kxr_main = desc_kxr.copy() 
desc_kxr_main['VariableName'] = "KNEE_" + desc_kxr_main['VariableName']

# kxr features are knee-specific, we append a copy to describe the other knee's 
desc_kxr_opposite = desc_kxr.copy() 
desc_kxr_opposite['VariableName'] = "OTHERKNEE_" + desc_kxr_opposite['VariableName']
desc_kxr_opposite['Attribute'] = "(OTHER KNEE) " + desc_kxr_opposite['Attribute']
 
desc_kxr = pd.concat([desc_kxr_main, desc_kxr_opposite])

desc_kxr.info()
desc_kxr.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 4 to 23
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   VariableName     40 non-null     object
 1   Attribute        40 non-null     object
 2   Use in Analysis  40 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.2+ KB


,VariableName,Attribute,Use in Analysis
4,KNEE_V00BARCDBU,BL/FU kXR reading (BU): barcode of image analyzed,0
5,KNEE_V00XROSFM,BL/FU kXR reading (BU): osteophytes (OARSI gra...,1
6,KNEE_V00XRSCFM,BL/FU kXR reading (BU): sclerosis (OARSI grade...,1
7,KNEE_V00XRCYFM,BL/FU kXR reading (BU): cysts (Grades 0-1) fem...,1
8,KNEE_V00XRJSM,BL/FU kXR reading (BU): joint space narrowing ...,1


In [14]:
# combine the descriptions 
desc = pd.concat([desc_clinical, desc_enrollees, desc_kxr])
desc.info()
desc.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1322 entries, 2 to 23
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   VariableName     1322 non-null   object 
 1   Attribute        1322 non-null   object 
 2   Use in Analysis  1322 non-null   float64
dtypes: float64(1), object(2)
memory usage: 41.3+ KB


,VariableName,Attribute,Use in Analysis
2,P01RXRKOA,SV:Right knee baseline radiographic OA (defini...,1.0
3,P01LXRKOA,SV:Left knee baseline radiographic OA (definit...,1.0
4,V00URINHR1,EV:Urine collection: time collected (first col...,0.0
5,V00URINHR2,EV:Urine collection: time collected (repeat co...,0.0
6,V00BLDHRS1,EV:Phlebotomy: time venipuncture completed (fi...,0.0


# Create side-aware features and other features 

In [15]:
def extract_feature_pairs(desc): 
    """ Attempts to identify and pair up knee-specific features """

    df = desc[desc['Use in Analysis']==1].copy()
    df['IsCandidate'] = np.where(
        df['Attribute'].str.contains('left') | df['Attribute'].str.contains('Left'), 1, 0)
    df['AttributeTrunc'] = df['Attribute'].str.split('.').str[-1]
    
    out = df.query("IsCandidate==1")\
        .rename(columns={'VariableName': 'VariableName_L', 
                         'Attribute': 'Attribute_L',
                         'AttributeTrunc': 'AttributeTrunc_L', 
                         'Use in Analysis': 'Use in Analysis_L'
                        })
    out['AttributeTrunc_R'] = out['AttributeTrunc_L']\
        .str.replace('left', 'right')\
        .str.replace('Left', 'Right')
    
    lookup_by_varname = df.set_index('AttributeTrunc')[['VariableName','Attribute', 'Use in Analysis']]
    out = out\
        .merge(lookup_by_varname, how='left', left_on='AttributeTrunc_R', right_index=True)\
        .rename(columns={'VariableName': 'VariableName_R', 
                         'Attribute': 'Attribute_R', 
                         'Use in Analysis': 'Use in Analysis_R'})
    out = out[out['VariableName_R'].notnull()]
    out = out.drop(['IsCandidate'], axis=1)

    # manually specify pairs that cannot be captured by our rule-based approach 
    manually_paired = [
        ('V00LLVERT', 'V00RLVERT'), ('V00LPWKPRV', 'V00RPWKPRV'), ('V00LEXP2CV', 'V00REXP2CV'),
        ('P01KRSL', 'P01KRSR'), ('P01SVXRLKR', 'P01SVXRRKR')
    ]
    manual_df = pd.DataFrame(manually_paired, columns=['VariableName_L', 'VariableName_R'])
    variable_attrs = df.set_index('VariableName')[['Attribute', 'AttributeTrunc']]
    manual_df = manual_df\
        .merge(variable_attrs, how='left', left_on='VariableName_L', right_index=True)\
        .merge(variable_attrs, how='left', left_on='VariableName_R', right_index=True, suffixes=['_L', '_R'])
    manual_df['Use in Analysis_L'] = 1
    manual_df['Use in Analysis_R'] = 1    
    
    out = pd.concat([out, manual_df])
    out = out.drop(['AttributeTrunc_L', 'AttributeTrunc_R'], axis=1)
    
    # exception pair: unlike other features these are derived from a single feature 
    out = out.append({'VariableName_L': 'P02KRS3CVL', 
                      'Attribute_L': 'Considered knee replacement on left knee',
                      'Use in Analysis_L': 1, 
                      'VariableName_R': 'P02KRS3CVR', 
                      'Attribute_R': 'Considered knee replacement on right knee',
                      'Use in Analysis_R': 1}, ignore_index = True)
    out = out.append({'VariableName_L': 'V00400PAINL', 
                      'Attribute_L': 'Left knee pain during 400-meter walk',
                      'Use in Analysis_L': 1, 
                      'VariableName_R': 'V00400PAINR', 
                      'Attribute_R': 'Right knee pain during 400-meter walk',
                      'Use in Analysis_R': 1}, ignore_index = True)
    
    return out


def append_additional_features(df, desc, feature_pairs):
    
    df = df.copy()
    desc = desc.copy()
    
    # manually create the left/right versions of single features 
    df['P02KRS3CVR'] = np.where(df['P02KRS3CV'].isin([1,3]), 1, 0)
    df['P02KRS3CVL'] = np.where(df['P02KRS3CV'].isin([2,3]), 1, 0)    
    df['V00400PAINR'] = np.where(df['V00400PAIN'].isin([1,3]), 1, 0)        
    df['V00400PAINL'] = np.where(df['V00400PAIN'].isin([2,3]), 1, 0)
    
    # append "side-aware" features to dataset df 
    for (fl, fr) in feature_pairs[['VariableName_L', 'VariableName_R']].to_records(index=False):
        df[f'KNEE_{fl}'] = np.where(df['SIDE']==1, df[fr], df[fl])
        df[f'OTHERKNEE_{fl}'] = np.where(df['SIDE']==1, df[fl], df[fr])
        
    # also append them to data dictionary 
    base_features = feature_pairs[['VariableName_L', 'Attribute_L', 'Use in Analysis_L']]
    base_features.columns = ['VariableName', 'Attribute', 'Use in Analysis']    
    knee_features = base_features.assign(
        VariableName=lambda x: 'KNEE_' + x['VariableName'])
    other_knee_features = base_features.assign(
        VariableName=lambda x: 'OTHERKNEE_' + x['VariableName'], 
        Attribute=lambda x: '(OTHER KNEE) ' + x['Attribute']
    )
    desc = pd.concat([desc, knee_features, other_knee_features])
    
    # engineer other features 
    df['P02RACE_IsBlack'] = np.where(df['P02RACE']==2, 1, 0)
    desc = desc.append({'VariableName': 'P02RACE_IsBlack', 'Attribute': 'P02RACE=Black', 'Use in Analysis': 1},
                       ignore_index = True)

    return df, desc

In [16]:
feature_pairs = extract_feature_pairs(desc)
feature_pairs.info()
feature_pairs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   VariableName_L     194 non-null    object 
 1   Attribute_L        194 non-null    object 
 2   Use in Analysis_L  194 non-null    float64
 3   VariableName_R     194 non-null    object 
 4   Attribute_R        194 non-null    object 
 5   Use in Analysis_R  194 non-null    float64
dtypes: float64(2), object(4)
memory usage: 9.2+ KB


,VariableName_L,Attribute_L,Use in Analysis_L,VariableName_R,Attribute_R,Use in Analysis_R
0,P01LXRKOA,SV:Left knee baseline radiographic OA (definit...,1.0,P01RXRKOA,SV:Right knee baseline radiographic OA (defini...,1.0
1,P01SVLKOST,SV:Left knee baseline x-ray: evidence of knee ...,1.0,P01SVRKOST,SV:Right knee baseline x-ray: evidence of knee...,1.0
2,P01SVLKJSM,SV:Left knee baseline x-ray: evidence of knee ...,1.0,P01SVRKJSM,SV:Right knee baseline x-ray: evidence of knee...,1.0
3,P01SVLKJSL,SV:Left knee baseline x-ray: evidence of knee ...,1.0,P01SVRKJSL,SV:Right knee baseline x-ray: evidence of knee...,1.0
4,P01SVLKMI,SV:Left knee baseline x-ray: evidence of knee ...,1.0,P01SVRKMI,SV:Right knee baseline x-ray: evidence of knee...,1.0


In [17]:
df1, desc1 = append_additional_features(df=df, desc=desc, feature_pairs=feature_pairs)

In [18]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8957 entries, 0 to 8956
Columns: 1725 entries, Knee_ID to P02RACE_IsBlack
dtypes: float64(1636), int64(68), object(21)
memory usage: 117.9+ MB


# Subset Eligible Features and Output to CSV

In [19]:
OUTPUT_DATA_FP = 'output/combined_clinical_data.csv'
OUTPUT_DESC_FP = 'output/combined_clinical_variable_desc.csv'

In [20]:
def get_features_above_nan_thresh(df, feature_cols, max_nan_pct=.1): 
    """ Filters feature_cols and returns only those with less than X portion of nans """
    
    df = df[feature_cols]
    
    # threshold of non-nulls 
    n = len(df)
    min_count = int(n*(1-max_nan_pct))
    
    # return cols that meet threshold 
    col_counts = df.count() 
    retain_cols = col_counts[col_counts >= min_count].index.tolist()
    
    return retain_cols 


def subset_eligible_features(df, desc, cohort, feature_pairs):
    """ Returns a list of features that are eligible to use in CatBoost """

    manual_exclude = ['V00SUPMG', 'P02KRS3CV', 'P01MRKCBCV', 'V00NNOSERV', 'P02KSURGCV', 'P02KRSCV'] #'P02RACE'
    id_cols = cohort.columns.tolist() 

    eligible_cols = desc[desc['Use in Analysis']==1]['VariableName'].unique().tolist()
    eligible_cols = [col for col in eligible_cols if (col not in id_cols) and (col not in manual_exclude)]
    eligible_cols = get_features_above_nan_thresh(df, feature_cols=eligible_cols)

    left_knee_feature_stems = feature_pairs['VariableName_L'].unique().tolist()
    right_knee_feature_stems = feature_pairs['VariableName_R'].unique().tolist()
    knee_feature_stems = get_features_above_nan_thresh(df1, feature_cols=left_knee_feature_stems)

    knee_features = [f'KNEE_{col}' for col in knee_feature_stems]
    other_knee_features = [f'OTHERKNEE_{col}' for col in knee_feature_stems]
    patient_features = [col for col in eligible_cols if col 
                        not in knee_features+other_knee_features+left_knee_feature_stems+right_knee_feature_stems]
    core_features = ['KNEE_V00XRKL', 'P01BMI', 'V00AGE', 'P02RACE', 'P02SEX', 'KNEE_P01INJL', 'KNEE_V00LKALNMT']

    all_features = patient_features + knee_features + other_knee_features

    print(f"Number of patient-level features: {len(patient_features)}")
    print(f"Number of knee features: {len(knee_features)}")
    print(f"Number of other knee features: {len(other_knee_features)}")
    print(f"Total number of features: {len(all_features)}")
    
    out_df = df[all_features + id_cols]
    
    return out_df, desc

In [21]:
df2, desc2 = subset_eligible_features(df=df1, desc=desc1, cohort=cohort, feature_pairs=feature_pairs)

Number of patient-level features: 519
Number of knee features: 83
Number of other knee features: 83
Total number of features: 685


In [22]:
df2.to_csv(OUTPUT_DATA_FP, index=False)
desc2.to_csv(OUTPUT_DESC_FP, index=False)